In [23]:
import os
import json
import numpy as np
import pandas as pd
from utils.fillPoseEstimation import customImputer, column_names

In [24]:
def read_json_files(folder_path):
    json_data = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r") as file:
                for line in file:
                    try:
                        line_data = json.loads(line)
                        json_data.append(line_data)
                    except json.JSONDecodeError:
                        print(f"Error parsing JSON line: {line}")
    
    return json_data


In [25]:
file_path = "Videos"
json_data = read_json_files(file_path)

file_path="Videos/Bill Gates The next outbreak Were not ready TED.mp4.openpifpaf.json"

with open(file_path, 'r') as file:
    for line in file:
        try:
            line_data = json.loads(line)
            json_data.append(line_data)
        except json.JSONDecodeError:
            print(f"Error parsing JSON line: {line}")

In [26]:
complete_body = []
partial_body = []
incomplete_body = []

In [27]:
for item in json_data:
    predictions = item["predictions"]
    for prediction in predictions:
        keypoints = prediction["keypoints"]
        bbox = prediction["bbox"]
        score = prediction["score"]
        category_id = prediction["category_id"]

        keypoints_arr = np.array(keypoints).reshape(-1, 3)
        c_values = keypoints_arr[:, 2]

        if np.any(c_values <= 0):
            keypoints_without_c = keypoints_arr[:, :2]
            if np.sum(c_values <= 0) <= 4:
                #imputed_keypoints = customImputer(keypoints_without_c)
                imputed_keypoints = np.where(keypoints_without_c <= 0.0, np.nan, keypoints_without_c)
                incomplete_body.append(imputed_keypoints.tolist())
            #else:
            #    incomplete_body.append(keypoints_without_c.tolist())
        else:
            keypoints_without_c = keypoints_arr[:, :2]
            complete_body.append(keypoints_without_c.tolist())    

In [28]:
incomplete_body = np.array(incomplete_body)
complete_body = np.array(complete_body)

In [29]:
reshape_incomplete_body = incomplete_body.reshape(-1,34)
reshape_complete_body = complete_body.reshape(-1,34)

In [30]:
df_complete_body = pd.DataFrame(reshape_complete_body, columns=column_names)
df_incomplete_body = pd.DataFrame(reshape_incomplete_body, columns=column_names)

In [31]:
df_incomplete_body

,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,760.04,147.16,766.50,131.97,NaN,NaN,806.83,125.62,NaN,NaN,...,834.05,411.38,870.09,571.67,751.94,593.48,NaN,NaN,NaN,NaN
1,754.39,143.97,760.70,127.58,751.42,131.15,801.28,121.88,NaN,NaN,...,821.20,397.39,835.22,575.93,767.07,564.40,NaN,NaN,NaN,NaN
2,804.96,129.87,813.34,114.82,802.74,116.46,854.67,110.33,NaN,NaN,...,906.11,407.92,814.86,529.73,889.21,559.46,NaN,NaN,NaN,NaN
3,808.82,129.36,816.95,115.47,NaN,NaN,857.62,111.08,NaN,NaN,...,907.70,402.76,819.18,526.64,886.18,552.75,NaN,NaN,NaN,NaN
4,811.09,128.60,819.17,114.27,NaN,NaN,859.90,111.57,NaN,NaN,...,899.29,400.77,809.69,548.02,888.41,567.71,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,264.26,251.87,265.97,249.19,261.93,249.54,269.77,249.17,258.29,249.65,...,260.60,299.52,NaN,NaN,261.55,330.24,NaN,NaN,NaN,NaN
114403,332.91,238.93,334.42,236.78,330.78,236.94,NaN,NaN,326.42,237.79,...,326.29,282.01,336.05,307.75,325.88,308.91,333.07,327.30,323.97,329.98
114404,334.58,239.43,335.88,237.21,332.27,237.36,NaN,NaN,328.03,238.13,...,327.28,283.02,336.12,309.79,326.18,309.83,332.72,329.95,323.83,332.95
114405,334.87,240.07,336.13,237.77,332.61,238.09,338.74,237.74,328.64,238.66,...,328.48,286.38,338.21,308.99,327.91,311.20,NaN,NaN,326.25,332.11


In [32]:
df_complete_body

,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,670.79,278.54,672.53,276.80,669.33,276.84,675.25,277.91,667.14,277.75,...,663.08,311.66,671.87,333.78,661.95,333.37,669.33,353.90,658.41,353.79
1,670.59,278.36,672.35,276.72,669.16,276.74,675.20,277.91,667.10,277.68,...,663.10,311.44,671.51,333.73,661.97,333.11,669.15,353.84,658.44,353.46
2,670.50,278.39,672.29,276.75,669.06,276.78,675.09,277.89,666.95,277.59,...,663.07,311.42,671.51,333.85,661.94,333.21,669.08,353.98,658.33,353.18
3,670.49,278.43,672.24,276.77,668.98,276.81,675.09,277.92,666.81,277.74,...,663.03,311.28,671.26,333.48,661.58,331.99,668.91,353.41,658.26,352.93
4,670.54,278.39,672.16,276.70,668.93,276.77,674.87,277.80,666.57,277.89,...,662.88,311.34,671.12,333.49,661.34,331.94,669.10,353.39,658.04,352.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26464,461.05,53.93,463.20,51.32,458.48,51.65,466.68,53.76,455.18,54.67,...,456.13,108.14,475.97,143.02,451.48,145.44,482.53,174.69,448.06,176.54
26465,461.10,53.88,463.24,51.28,458.54,51.61,466.67,53.75,455.18,54.67,...,456.14,108.12,475.98,143.02,451.46,145.41,482.58,174.90,448.03,176.59
26466,461.09,53.89,463.25,51.28,458.53,51.61,466.69,53.75,455.19,54.67,...,456.12,108.15,475.99,143.01,451.49,145.39,482.64,174.85,448.03,176.58
26467,461.09,53.90,463.24,51.29,458.53,51.62,466.68,53.77,455.18,54.67,...,456.07,108.18,475.98,143.00,451.52,145.43,482.56,174.78,448.09,176.57


In [33]:
df_complete_body.to_csv('data/complete_body.csv', index=False)
df_incomplete_body.to_csv('data/incomplete_body.csv', index=False)